In [5]:
from sqlalchemy import create_engine, inspect
import pandas as pd

In [6]:
engine = create_engine("sqlite:///../db/mlsseasons.sqlite", echo=False)

In [7]:
inspect(engine)

In [8]:
dataset=pd.DataFrame(engine.execute("""
select s."First Name", s."Last Name", s.Position,
c.Club, c.pts, c."#" as standing, c.season
from seasons c
join club_map m on m.long_name = c.Club
join salaries s on s."Club (grouped)" = m.short_name
where c.season = s.Season
"""),columns=['First Name','Last Name','Position','Club','PTS','Standing','Season'])
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season
0,None,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007
2,Brad,North,F,DCD.C. United,55,1,2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007
...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018


In [9]:
dataset["First Name"] = dataset["First Name"].fillna("")

In [10]:
dataset['Player'] = dataset['First Name'] + dataset['Last Name'] # + dataset['Position']
dataset['Team'] = dataset['Club'] + dataset['Season'].apply(str)
dataset

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team
0,,Carreiro da Silva Fred,M-F,DCD.C. United,55,1,2007,Carreiro da Silva Fred,DCD.C. United2007
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007
2,Brad,North,F,DCD.C. United,55,1,2007,BradNorth,DCD.C. United2007
3,Brian,Carroll,M,DCD.C. United,55,1,2007,BrianCarroll,DCD.C. United2007
4,Bryan,Arguez,M-D,DCD.C. United,55,1,2007,BryanArguez,DCD.C. United2007
...,...,...,...,...,...,...,...,...,...
6206,Quincy,Amarikwa,F,SJSan Jose Earthquakes,21,23,2018,QuincyAmarikwa,SJSan Jose Earthquakes2018
6207,Shea,Salinas,M,SJSan Jose Earthquakes,21,23,2018,SheaSalinas,SJSan Jose Earthquakes2018
6208,Tommy,Thompson,M,SJSan Jose Earthquakes,21,23,2018,TommyThompson,SJSan Jose Earthquakes2018
6209,"Valeri ""Vako""",Qazaishvili,M,SJSan Jose Earthquakes,21,23,2018,"Valeri ""Vako""Qazaishvili",SJSan Jose Earthquakes2018


In [25]:
dataset.to_csv("Tableau_data.csv", encoding='utf-8', index=False)

In [15]:
a.keys()

NameError: name 'a' is not defined

In [16]:
dataset.groupby("Player").count()["Season"].describe()

count    2250.000000
mean        2.758667
std         2.360747
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        12.000000
Name: Season, dtype: float64

In [17]:
y = dataset["Player"].tolist()

In [18]:
 from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y

array([ 384,  231,  279, ..., 2122, 2165, 2222], dtype=int64)

In [19]:
dataset['encoded_player'] = encoded_y
dataset['present'] = 1.0
dataset.loc[dataset['Player'] == 'BenOlsen']

,First Name,Last Name,Position,Club,PTS,Standing,Season,Player,Team,encoded_player,present
1,Ben,Olsen,M,DCD.C. United,55,1,2007,BenOlsen,DCD.C. United2007,231,1.0
627,Ben,Olsen,M,DCD.C. United,38,10,2008,BenOlsen,DCD.C. United2008,231,1.0
1005,Ben,Olsen,M,DCD.C. United,40,10,2009,BenOlsen,DCD.C. United2009,231,1.0


In [73]:
samples = dataset.pivot(index="Team", columns="encoded_player", values='present')
samples

encoded_player,0,1,2,3,4,5,6,7,8,9,...,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250
Team,,,,,,,,,,,,,,,,,,,,,
ATLAtlanta United FC2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATLAtlanta United FC2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHIChicago Fire FC2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VANVancouver Whitecaps FC2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VANVancouver Whitecaps FC2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
samples.describe()

NameError: name 'samples' is not defined

In [21]:
dataset['Standing'] = dataset['Standing'].astype(int)
dataset['PTS'] = dataset['PTS'].astype(int)
y = dataset.groupby('Team').mean()['Standing'].sort_index()
y

Team
ATLAtlanta United FC2017          4
ATLAtlanta United FC2018          2
CHIChicago Fire FC2007            8
CHIChicago Fire FC2008            3
CHIChicago Fire FC2009            6
                                 ..
VANVancouver Whitecaps FC2014     9
VANVancouver Whitecaps FC2015     3
VANVancouver Whitecaps FC2016    16
VANVancouver Whitecaps FC2017     8
VANVancouver Whitecaps FC2018    14
Name: Standing, Length: 218, dtype: int32

In [22]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets

ModuleNotFoundError: No module named 'tensorflow'

In [23]:
X = samples.fillna(0).sort_index()

NameError: name 'samples' is not defined

In [120]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [121]:
X_test.shape

(55, 2251)

In [126]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[-1]))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 32)                72064     
_________________________________________________________________
dense_21 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_22 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 9         
Total params: 72,737
Trainable params: 72,737
Non-trainable params: 0
_________________________________________________________________


In [127]:
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [128]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
6/6 - 0s - loss: 123.7152 - accuracy: 0.0000e+00
Epoch 2/100
6/6 - 0s - loss: 120.9275 - accuracy: 0.0000e+00
Epoch 3/100
6/6 - 0s - loss: 118.0542 - accuracy: 0.0123
Epoch 4/100
6/6 - 0s - loss: 114.2574 - accuracy: 0.0429
Epoch 5/100
6/6 - 0s - loss: 109.0280 - accuracy: 0.0552
Epoch 6/100
6/6 - 0s - loss: 101.6678 - accuracy: 0.0613
Epoch 7/100
6/6 - 0s - loss: 92.4238 - accuracy: 0.0613
Epoch 8/100
6/6 - 0s - loss: 81.2518 - accuracy: 0.0613
Epoch 9/100
6/6 - 0s - loss: 68.5782 - accuracy: 0.0613
Epoch 10/100
6/6 - 0s - loss: 55.5454 - accuracy: 0.0613
Epoch 11/100
6/6 - 0s - loss: 43.2998 - accuracy: 0.0613
Epoch 12/100
6/6 - 0s - loss: 33.2201 - accuracy: 0.0613
Epoch 13/100
6/6 - 0s - loss: 25.5361 - accuracy: 0.0613
Epoch 14/100
6/6 - 0s - loss: 20.6469 - accuracy: 0.0613
Epoch 15/100
6/6 - 0s - loss: 17.2138 - accuracy: 0.0613
Epoch 16/100
6/6 - 0s - loss: 14.1681 - accuracy: 0.0613
Epoch 17/100
6/6 - 0s - loss: 11.9324 - accuracy: 0.0613
Epoch 18/100
6/6 - 0s - lo

In [129]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 33.1486 - accuracy: 0.0364
Normal Neural Network - Loss: 33.14860534667969, Accuracy: 0.036363635212183
